In [ ]:
from pyspark.sql.functions import col, lit

# ╔════════════════════════════════════════════════════════╗
# ║ 1️⃣  Connexion au compte de stockage Azure              ║
# ╚════════════════════════════════════════════════════════╝
dbutils.widgets.text("AZURE_STORAGE_KEY", "")
dbutils.widgets.text("SQL_PASSWORD", "")
storage_account = "datalakemedallion"
storage_key = dbutils.widgets.get("AZURE_STORAGE_KEY")
container_name = "silver"
folder_path = "transformations/transactions_trsf/part-00000-tid-7307336381776231701-9e7de5cb-4c94-439c-b9d0-39ad951b34d4-19-1.c000.snappy.parquet"

spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", storage_key)

source_path = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/{folder_path}"

# ╔════════════════════════════════════════════════════════╗
# ║ 2️⃣  Lecture du fichier Parquet contenant les transactions ║
# ╚════════════════════════════════════════════════════════╝

df_transactions = spark.read.parquet(source_path)
display(df_transactions)

# ╔════════════════════════════════════════════════════════╗
# ║ 3️⃣  Connexion à la base SQL Azure                      ║
# ╚════════════════════════════════════════════════════════╝

jdbc_hostname = "bank-srv.database.windows.net"
jdbc_port = 1433
database_name = "BankDB"
username = "adminsql"
password = dbutils.widgets.get("SQL_PASSWORD")

jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:{jdbc_port};database={database_name};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
table_name = "clients"

df_sql = (
    spark.read.format("jdbc")
    .option("url", jdbc_url)
    .option("dbtable", table_name)
    .option("user", username)
    .option("password", password)
    .load()
)

display(df_sql)

# ╔════════════════════════════════════════════════════════╗
# ║ 4️⃣  Contrôle qualité : identification des anomalies     ║
# ╚════════════════════════════════════════════════════════╝

valid_currencies = ['EUR', 'CAD', 'USD', 'MAD']
df_customers_ids = df_sql.select("customer_id").withColumnRenamed("customer_id", "cust_id_ref")

# Join avec la base clients
df_joined = df_transactions.join(
    df_customers_ids,
    df_transactions.customer_id == df_customers_ids.cust_id_ref,
    how="left"
)

# 🔸 Transactions invalides
df_invalid = df_joined.filter(
    (col("amount") <= 0) |
    (~col("currency").isin(valid_currencies)) |
    (col("transaction_type") == "inconnu") |
    (col("status") == "inconnu") |
    (col("channel") == "inconnu") |
    (col("merchant_category") == "inconnu") |
    (col("transaction_date") > lit("2025-10-31")) |
    (col("cust_id_ref").isNull())  # client absent
).drop("cust_id_ref")

# 🔸 Transactions valides (toutes les autres)
df_valid = df_joined.filter(
    ~(
        (col("amount") <= 0) |
        (~col("currency").isin(valid_currencies)) |
        (col("transaction_type") == "Inconnu") |
        (col("status") == "Inconnu") |
        (col("channel") == "Inconnu") |
        (col("merchant_category") == "Inconnu") |
        (col("transaction_date") > lit("2025-10-31")) |
        (col("cust_id_ref").isNull())|
        (col("account_id") == "Inconnu") |
        (col("transaction_id") == "Inconnu") 
    )
).drop("cust_id_ref")

print("✅ Nombre de transactions invalides :", df_invalid.count())
print("✅ Nombre de transactions valides :", df_valid.count())

In [ ]:
valid_path = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/control_qualite/valid_data/"
invalid_path = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/control_qualite/invalid_data/"

df_valid.write.mode("overwrite").parquet(valid_path)
df_invalid.write.mode("overwrite").parquet(invalid_path)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import *

# ✨ Exemple simple — DataFrame transactions
data_transactions = [
    ("T001", "C001", "A001", 150.0, "USD", "2025-01-10", "online", "payment", "valid"),
    ("T002", "C002", "A002", -20.0, "MAD", "2025-02-02", "atm", "withdrawal", "valid"),   # montant négatif ❌
    ("T003", "C999", "A003", 300.0, "EURO", "2025-03-01", "store", "payment", "valid"),  # currency invalide ❌
    ("T004", "C001", "A004", 500.0, "USD", "2026-01-01", "store", "payment", "valid"),   # date future ❌
]

schema_transactions = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("account_id", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("transaction_date", StringType(), True),
    StructField("channel", StringType(), True),
    StructField("transaction_type", StringType(), True),
    StructField("status", StringType(), True)
])

df_transactions = spark.createDataFrame(data_transactions, schema_transactions)
display(df_transactions)
silver_output_path = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/test/"
df_transactions.write.mode("overwrite").parquet(silver_output_path)
print("✅ DataFrame enregistré avec succès dans silver/test/")
print("Fin")

In [ ]:
print("hello")